In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
tmdb = pd.read_csv('../../Downloads/raw_tmdb.csv')
tmdb.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,160000000,https://www.warnerbros.com/movies/inception,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,165000000,http://www.interstellarmovie.net/,tt0816692,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,185000000,https://www.warnerbros.com/movies/dark-knight/,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,237000000,https://www.avatar.com/movies/avatar,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,220000000,https://www.marvel.com/movies/the-avengers,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [4]:
# removed movies that are unreleased
tmdb = tmdb[tmdb['status'] == 'Released']
# remove movies that do not have a description
tmdb = tmdb[tmdb['overview'].notna()]
# change dates to pandas datetime format
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'])
# original language is english
tmdb = tmdb[tmdb['original_language'] == 'en']
# drop columns
tmdb = tmdb.drop(columns = ['backdrop_path', 'homepage', 'original_title', 'poster_path', 'tagline', 'spoken_languages', 'original_language', 'status'])
# insert release_year
years = tmdb['release_date'].dt.year
tmdb.insert(4, 'release_year', years)
# insert profit
tmdb.insert(9, 'profit', tmdb['revenue'] - tmdb['budget'])
# remove 0 budget movies
tmdb = tmdb[tmdb['budget'] != 0]

In [5]:
# drop movies without title
tmdb = tmdb[tmdb['title'].notna()]
# drop movies without runtimes
tmdb = tmdb[tmdb['runtime'] != 0]
# drop movies without release date
tmdb = tmdb[tmdb['release_date'].notna()]
# convert release_year to int
tmdb['release_year'] = tmdb['release_year'].astype(int)
# rename popularity to tmdb_popularity
tmdb = tmdb.rename(columns={'popularity': 'tmdb_popularity'})

In [6]:
# reset index
tmdb = tmdb.reset_index(drop=True)

In [7]:
tmdb_rev = tmdb[tmdb['revenue'] != 0]
tmdb_rev.to_csv('./data/tmdb_rev.csv')

In [8]:
# import pandas as pd
# import os
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, r2_score
# from lightgbm import LGBMRegressor
# from sentence_transformers import SentenceTransformer
# import pickle
# from sklearn.decomposition import PCA

# # --- Load and preprocess data ---
# print("Loading data...")
# df = pd.read_csv('./data/tmdb_rev.csv')
# df = df.dropna(subset=['overview', 'budget', 'release_year', 'revenue', 'title'])

# df['overview'] = df['overview'].fillna("")
# df['budget'] = df['budget'].fillna(0)
# df['release_year'] = df['release_year'].fillna(0)
# df['revenue'] = df['revenue'].fillna(0)

# # --- Get overview embeddings in batches ---
# print("Generating embeddings...")
# bert = SentenceTransformer('all-MiniLM-L6-v2')
# batch_size = 100
# overview_embeddings = []
# for i in range(0, len(df), batch_size):
#     batch = df['overview'].iloc[i:i+batch_size].tolist()
#     overview_embeddings.append(bert.encode(batch, show_progress_bar=True))
# overview_embeddings = np.vstack(overview_embeddings)

# # Reduce dimensionality
# print("Reducing dimensions...")
# pca = PCA(n_components=100)
# overview_embeddings = pca.fit_transform(overview_embeddings)

# # --- Features ---
# X = np.hstack([
#     overview_embeddings,
#     df[['budget', 'release_year']].values
# ])
# y = np.log1p(df['revenue'].values)  # Log transform for better performance

# print(f"Final data shape - X: {X.shape}, y: {y.shape}")

# # --- Split & train ---
# print("Training model...")
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model = LGBMRegressor(
#     n_estimators=200,
#     random_state=42,
#     max_depth=5,
#     num_leaves=20,
#     subsample=0.8,
#     colsample_bytree=0.8
# )
# model.fit(X_train, y_train)

# # --- Make predictions ---
# y_pred = model.predict(X_test)

# # --- Evaluate the model ---
# mae = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
# r2 = r2_score(y_test, y_pred)

# # --- Print the results ---
# print(f"Mean Absolute Error (MAE): ${mae:,.2f}")
# print(f"R-squared (R²): {r2:.4f}")

# # --- Save the model and embeddings ---
# os.makedirs("models", exist_ok=True)

# with open("models/revenue_model.pkl", "wb") as f:
#     pickle.dump(model, f)

# with open("models/bert_model.pkl", "wb") as f:
#     pickle.dump(bert, f)

# df[['title', 'overview']].to_csv("models/title_reference.csv", index=False)
# np.save("models/overview_embeddings.npy", overview_embeddings)

# print("✅ Model training complete and saved.")

In [9]:
# import pandas as pd
# import os
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, r2_score
# from lightgbm import LGBMRegressor
# from sentence_transformers import SentenceTransformer
# import pickle
# from sklearn.decomposition import PCA
# import gc  # Garbage collector

# def safe_embedding_generation(texts, model, batch_size=50):
#     """Generate embeddings with memory safety"""
#     embeddings = []
#     for i in range(0, len(texts), batch_size):
#         batch = texts[i:i+batch_size]
#         embeddings.append(model.encode(batch, show_progress_bar=False))
#         gc.collect()  # Force garbage collection
#     return np.vstack(embeddings)

# def main():
#     # --- Load and preprocess data ---
#     print("Loading data...")
#     try:
#         df = pd.read_csv('./data/tmdb_rev.csv')
#         df = df.dropna(subset=['overview', 'budget', 'release_year', 'revenue', 'title'])
        
#         # Sample data if too large (comment out for full dataset)
#         df = df.sample(5000, random_state=42) if len(df) > 10000 else df
        
#         df['overview'] = df['overview'].fillna("")
#         df['budget'] = df['budget'].fillna(0).astype(np.float32)
#         df['release_year'] = df['release_year'].fillna(0).astype(np.int16)
#         df['revenue'] = df['revenue'].fillna(0).astype(np.float32)
#     except Exception as e:
#         print(f"Data loading failed: {str(e)}")
#         return

#     # --- Get overview embeddings in safe batches ---
#     print("Generating embeddings...")
#     try:
#         bert = SentenceTransformer('all-MiniLM-L6-v2')
#         overview_embeddings = safe_embedding_generation(df['overview'].tolist(), bert, batch_size=50)
        
#         # Reduce dimensionality aggressively
#         print("Reducing dimensions...")
#         pca = PCA(n_components=50)  # More aggressive reduction
#         overview_embeddings = pca.fit_transform(overview_embeddings).astype(np.float32)
#     except Exception as e:
#         print(f"Embedding generation failed: {str(e)}")
#         return

#     # --- Features with memory optimization ---
#     try:
#         X = np.hstack([
#             overview_embeddings,
#             df[['budget', 'release_year']].values
#         ]).astype(np.float32)
#         y = np.log1p(df['revenue'].values).astype(np.float32)
#         y = y.dropna()

#         print(f"Final data shape - X: {X.shape}, y: {y.shape}")
#         print(f"Approximate memory usage: {X.nbytes / (1024**2):.2f} MB")
#     except Exception as e:
#         print(f"Feature preparation failed: {str(e)}")
#         return

#     # --- Split & train with memory constraints ---
#     print("Training model...")
#     try:
#         X_train, X_test, y_train, y_test = train_test_split(
#             X, y, test_size=0.2, random_state=42
#         )
        
#         # Free memory
#         del X, y, overview_embeddings, df
#         gc.collect()

#         model = LGBMRegressor(
#             n_estimators=100,  # Reduced number
#             random_state=42,
#             max_depth=4,       # More shallow trees
#             num_leaves=15,    # Fewer leaves
#             subsample=0.7,    # Smaller fraction of data
#             colsample_bytree=0.7,
#             verbose=-1        # Silent training
#         )
        
#         model.fit(X_train, y_train)
        
#         # --- Make predictions ---
#         y_pred = model.predict(X_test)
        
#         # --- Evaluate the model ---
#         mae = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
#         r2 = r2_score(y_test, y_pred)
        
#         print(f"Mean Absolute Error (MAE): ${mae:,.2f}")
#         print(f"R-squared (R²): {r2:.4f}")
#     except Exception as e:
#         print(f"Model training failed: {str(e)}")
#         return

#     # --- Save the model and embeddings ---
#     try:
#         os.makedirs("models", exist_ok=True)
        
#         # Save models with protocol=4 for better compatibility
#         with open("models/revenue_model.pkl", "wb") as f:
#             pickle.dump(model, f, protocol=4)
            
#         with open("models/bert_model.pkl", "wb") as f:
#             pickle.dump(bert, f, protocol=4)
            
#         with open("models/pca_model.pkl", "wb") as f:
#             pickle.dump(pca, f, protocol=4)
        
#         # Save smaller reference data
#         pd.DataFrame({
#             'title': df['title'].iloc[:1000],  # Only save first 1000
#             'overview': df['overview'].iloc[:1000]
#         }).to_csv("models/title_reference.csv", index=False)
        
#         # Save reduced embeddings
#         np.save("models/overview_embeddings.npy", overview_embeddings[:1000])  # Only first 1000
        
#         print("✅ Model training complete and saved.")
#     except Exception as e:
#         print(f"Model saving failed: {str(e)}")

# if __name__ == "__main__":
#     main()

In [10]:
%pip install gensim
%pip install nltk
%pip install xgboost
%pip install cloudpickle

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Improved Classification Pipeline Using GloVe Embeddings + Tuned XGBoost + Enhanced Features
import pandas as pd
import numpy as np
import ast
import gensim.downloader as api
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from xgboost import XGBClassifier

df = pd.read_csv('./data/tmdb_rev.csv')
df = df[df["budget"] != 0]
# removing the genres that are not strings
df = df[df['genres'].apply(lambda x: isinstance(x, str))]

# Step 1: Convert genre strings into lists
list_genres = df["genres"].dropna().apply(lambda x: [g.strip() for g in x.split(",")])

# Step 2: Flatten and collect unique genres
unique_genres = set(genre for sublist in list_genres for genre in sublist)

# Optional: Sort alphabetically
unique_genres = sorted(unique_genres)

selected_genres = ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Romance', 'Science Fiction', 'Thriller']

# Parse genres to list
df['genre_list'] = df['genres'].apply(lambda x: [g.strip() for g in x.split(',') if g.strip() in selected_genres])


nltk.download('vader_lexicon')

# ---------- Load GloVe ----------
print("Loading GloVe embeddings...")
glove_model = api.load("glove-wiki-gigaword-100")

# ---------- Custom Transformers ----------
class ListToWordsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

class TextToWordList(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.fillna("").apply(lambda x: x.lower().split())

class PretrainedEmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, vector_size):
        self.model = model
        self.vector_size = vector_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        result = []
        for item in X:
            vectors = [self.model[word] for word in item if word in self.model]
            avg_vector = np.mean(vectors, axis=0) if vectors else np.zeros(self.vector_size)
            result.append(avg_vector)
        return np.array(result)

class SentimentExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.fillna("").apply(lambda x: self.analyzer.polarity_scores(x)['compound']).to_frame()

# ---------- Data Preparation ----------

# Bin revenue into categories (e.g., flop, average, hit, blockbuster)
revenue_bins = [0, 1e7, 5e7, 1e8, np.inf]
bin_labels = [0, 1, 2, 3]
df['revenue_class'] = pd.cut(df['revenue'], bins=revenue_bins, labels=bin_labels)
df = df.dropna(subset=['revenue_class'])
y = df['revenue_class'].astype(int)

# Features
X = df[['runtime', 'adult', 'budget', 'overview', 'genre_list', 'production_companies', 'production_countries']]

# ---------- Train/Test Split ----------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ---------- GloVe & Sentiment Pipelines ----------
overview_glove = Pipeline([
    ('text_to_words', TextToWordList()),
    ('glove_embed', PretrainedEmbeddingTransformer(glove_model, 100))
])

genre_glove = Pipeline([
    ('list_to_words', ListToWordsTransformer()),
    ('glove_embed', PretrainedEmbeddingTransformer(glove_model, 100))
])

sentiment_pipe = Pipeline([
    ('sentiment', SentimentExtractor())
])

# ---------- Multi-hot Pipelines for company/country ----------
from sklearn.preprocessing import MultiLabelBinarizer

class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        parsed = X.apply(self._safe_parse)
        self.mlb.fit(parsed)
        return self

    def transform(self, X):
        parsed = X.apply(self._safe_parse)
        return self.mlb.transform(parsed)

    def _safe_parse(self, x):
        if isinstance(x, list):
            return x
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except (ValueError, SyntaxError):
                return [item.strip() for item in x.split(",") if item.strip()]
        return []

company_pipe = Pipeline([
    ('mlb', MultiLabelBinarizerTransformer())
])

country_pipe = Pipeline([
    ('mlb', MultiLabelBinarizerTransformer())
])

# ---------- Preprocessing ColumnTransformer ----------
preprocessor = ColumnTransformer(transformers=[
    ('runtime', StandardScaler(), ['runtime']),
    ('budget', StandardScaler(), ['budget']),
    ('adult', OneHotEncoder(drop='if_binary'), ['adult']),
    ('overview_glove', overview_glove, 'overview'),
    ('genre_glove', genre_glove, 'genre_list'),
    ('companies', company_pipe, 'production_companies'),
    ('countries', country_pipe, 'production_countries'),
    ('sentiment', sentiment_pipe, 'overview')
])

# ---------- Full Model Pipeline ----------
model = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.85,
        colsample_bytree=0.85,
        use_label_encoder=False,
        eval_metric='mlogloss',
        random_state=42
    ))
])

# ---------- Train & Evaluate ----------
print("Training model...")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Flop', 'Average', 'Hit', 'Blockbuster']))

import cloudpickle
import os

os.makedirs("models", exist_ok=True)

with open("models/final_model.pkl", "wb") as f:
    cloudpickle.dump(model, f)

print("✅ Model saved using cloudpickle at models/final_model.pkl")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lasyayadlapati/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Loading GloVe embeddings...
Training model...


/Users/lasyayadlapati/miniforge3/envs/dsc80/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:10:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/lasyayadlapati/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/Users/lasyayadlapati/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['100 Feet Productions', '1212 Entertainment', '2 Bridges Productions', '2 Loop Films', '23ten', '321 Films', '3Mark Entertainment', '3ality Digital Entertainment', '7 Bucks Entertainment', '88 Rising', 'A Loopy Production LLC', 'A Steven Haft Production', 


Accuracy: 0.6569506726457399

Classification Report:
              precision    recall  f1-score   support

        Flop       0.78      0.87      0.82       823
     Average       0.46      0.55      0.50       440
         Hit       0.22      0.03      0.05       190
 Blockbuster       0.65      0.63      0.64       331

    accuracy                           0.66      1784
   macro avg       0.53      0.52      0.50      1784
weighted avg       0.62      0.66      0.63      1784

✅ Model saved using cloudpickle at models/final_model.pkl


In [12]:
%pip install sentence_transformers
%pip install transformers
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
import torch

# Load models
title_gen_model_name = "gpt2"
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"

# GPT-2 tokenizer & model for title generation
tokenizer = GPT2Tokenizer.from_pretrained(title_gen_model_name)
model = GPT2LMHeadModel.from_pretrained(title_gen_model_name)
model.eval()

# SentenceTransformer for semantic awareness (optional here but useful if you expand this)
embedder = SentenceTransformer(embed_model_name)

def generate_title_from_overview(overview, budget=None, release_year=None, max_length=12):
    # Create a simple prompt
    prompt = f"Movie Overview: {overview}\nSuggested Title:"
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]

    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=input_ids.shape[1] + max_length,
            temperature=0.9,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode and clean output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    title = generated_text.split("Suggested Title:")[-1].strip().split("\n")[0]
    
    return title

In [14]:
df = df[df["budget"] != 0]
# removing the genres that are not strings
df = df[df['genres'].apply(lambda x: isinstance(x, str))]

# Step 1: Convert genre strings into lists
list_genres = df["genres"].dropna().apply(lambda x: [g.strip() for g in x.split(",")])

# Step 2: Flatten and collect unique genres
unique_genres = set(genre for sublist in list_genres for genre in sublist)

# Optional: Sort alphabetically
unique_genres = sorted(unique_genres)


# Two-Stage Pipeline: Predict Genre, then Predict Revenue using Genre + Emotion + Synopsis

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics import mean_absolute_error, f1_score, accuracy_score, hamming_loss
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from xgboost import XGBRegressor

# ---------------------
# 1. Load and Clean Data
# ---------------------
# Select target genres to simplify
selected_genres = ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Romance', 'Science Fiction', 'Thriller']

# Parse genres to list
df['genre_list'] = df['genres'].apply(lambda x: [g.strip() for g in x.split(',') if g.strip() in selected_genres])

# ---------------------
# 2. Multi-label Genre Matrix
# ---------------------
mlb = MultiLabelBinarizer(classes=selected_genres)
Y_genres = mlb.fit_transform(df['genre_list'])

# ---------------------
# 3. Extract Sentiment Feature
# ---------------------
analyzer = SentimentIntensityAnalyzer()
df['sentiment'] = df['overview'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])

# ---------------------
# 4. TF-IDF of Synopsis
# ---------------------
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X_synopsis = tfidf.fit_transform(df['overview']).toarray()

# ---------------------
# 5. Stage 1: Genre Prediction
# ---------------------
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_synopsis, Y_genres, test_size=0.2, random_state=42)

genre_model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
genre_model.fit(X_train_g, y_train_g)
genre_preds = genre_model.predict(X_synopsis)  # Use full set for input to Stage 2

# ---------------------
# 6. Combine Features for Revenue Model
# ---------------------
metadata_cols = ['budget', 'release_year']
metadata = df[metadata_cols].fillna(0)
scaler = StandardScaler()
X_metadata = scaler.fit_transform(metadata)

X_emotion = df[['sentiment']].values  # Only sentiment for now
X_combined = np.hstack([genre_preds, X_emotion, X_synopsis, X_metadata])

# ---------------------
# 7. Revenue Prediction
# ---------------------
y_revenue = df['revenue'].values
y_log = np.log1p(y_revenue)

X_train_rev, X_test_rev, y_train_rev, y_test_rev = train_test_split(X_combined, y_log, test_size=0.2, random_state=42)

revenue_model = XGBRegressor()
revenue_model.fit(X_train_rev, y_train_rev)

# Predict and evaluate
log_preds = revenue_model.predict(X_test_rev)
revenue_preds = np.expm1(log_preds)
revenue_true = np.expm1(y_test_rev)

revenue_acc = revenue_model.score(X_test_rev, y_test_rev)
print("Revenue Accuracy: ", revenue_acc)
print("\n💰 Revenue Prediction Results:")
print("MAE:", mean_absolute_error(revenue_true, revenue_preds))

In [ ]:
import joblib
joblib.dump(genre_model, "models/genre_model.pkl")
joblib.dump(revenue_model, "models/revenue_model.pkl")
joblib.dump(tfidf, "models/tfidf_vectorizer.pkl")
joblib.dump(scaler, "models/scaler.pkl")
joblib.dump(mlb, "models/mlb.pkl")